In [4]:
# always get fresh copies!
! cp ~cs186/sp16/hw2/part2/part2.db .
! chmod 700 part2.db
! cp ~cs186/sp16/hw2/part2/part2sampled.db .
! chmod 700 part2sampled.db
! cp ~cs186/sp16/hw2/part2/part2test.db .
! chmod 700 part2test.db

In [5]:
import subprocess, sqlite3, csv, os, sys

# jupyter weirdness; this makes sure that stdout is not buffered
def printNow(s):
    print s
    sys.stdout.flush()

    
def checkFile(f):
    if not (os.path.isfile(f)):
        raise IOError("DB File " + f + " is not here!")


currentDB1 = currentDB2 = "part2sampled.db"

# for grading dataset
# note that the two DB are different! currentDB1 is smaller in size
if ('CS186GRADER' in os.environ):
    currentDB1 = "part2sampled.db" 
    currentDB2 = "part2.db" 
    
checkFile(currentDB1)
dbQ1 = sqlite3.connect(currentDB1) 
dbQ1.text_factory = str

checkFile(currentDB2)
dbQ2 = sqlite3.connect(currentDB2)
dbQ2.text_factory = str

In [6]:
# Q1
# Degrees of separation
# Expected output: a paths table (as defined), note that we will be checking the original
#                  paths, and not just the average value

def initDegrees():
    dbQ1.executescript("""
        -- create paths table
        DROP TABLE IF EXISTS paths;
        DROP TABLE IF EXISTS pathsLastUpdated;
        DROP INDEX IF EXISTS paths_src_idx;
        DROP INDEX IF EXISTS paths_dst_idx;

        CREATE TABLE paths (
            src varchar(200),
            dst varchar(200),
            length integer,
            PRIMARY KEY (src, dst)
        );
        
        CREATE TABLE pathsLastUpdated (
            src varchar(200),
            dst varchar(200),
            length integer
        );

        CREATE INDEX paths_src_idx ON paths(src);
        CREATE INDEX paths_dst_idx ON paths(dst);
    """);
    dbQ1.commit()
    return


def populatePaths():
    dbQ1.executescript("""
      -- initialize paths with the first level connections
      INSERT INTO paths SELECT
        src,
        dst,
        1
      FROM links;      
      
      -- initialized for your convenience the "last udpated"
      INSERT INTO pathsLastUpdated SELECT
        src,
        dst,
        1
      FROM links;
    """);
    
    dbQ1.commit()
    return;


# Please implement the algorithm such that you don't repeat paths that were previously explored
# For example, for graph a->b->c->d, we want to see (a,b), (b,c), (c,d) then (a,c), (b,d), then (a,d)
# This makes a rather significant performance difference. To this end we have set up the 
#   pathsLastUpdated for you to store intermediate results
def getNewPaths():
    dbQ1.executescript("""
        -- this is an intermediate table for you to store temp data
        DROP TABLE IF EXISTS pathsNew;  
        CREATE TABLE pathsNew (
            src varchar(200),
            dst varchar(200),
            length integer
        );    
        
            INSERT INTO pathsNew        
            -- TODO (hint: use pathsLastUpdated)
           
           SELECT * 
           FROM(
           
            SELECT pathsLastUpdated.src, links.dst, pathsLastUpdated.length+1
            FROM pathsLastUpdated
            INNER JOIN links
            ON pathsLastupdated.dst = links.src
            
            )
            WHERE src <> dst
            
        ;
     """);
    dbQ1.commit()
    return

def updatePathsWithNewpaths():
    dbQ1.executescript("""
        DROP TABLE IF EXISTS pathsLastUpdated;
        
        -- add it to path
        INSERT INTO paths
            -- TODO (hint: use pathsNew)
            

        SELECT DISTINCT * 
        FROM pathsNew p
        WHERE NOT EXISTS (SELECT 1 
                          FROM paths X 
                          WHERE p.src = X.src AND 
                          p.dst = x.dst)





        ;
        
        ALTER TABLE pathsNew RENAME TO pathsLastUpdated;        
    """);
    dbQ1.commit()
    return


def getPathsCount():
    r = dbQ1.execute("SELECT COUNT(*) FROM paths;").fetchone()
    return r[0]


def getAveragePathsLength():
    r = dbQ1.execute("SELECT avg(length) FROM paths;").fetchone()
    return r[0]


def findDegreeDistribution():
    printNow("initializing degrees.  Calling sqlite...")
    initDegrees()
    printNow("Done, now populating paths.  Calling sqlite...")
    populatePaths()
    printNow("Done, now getting pathcount.  Calling sqlite...")
    oldCount = getPathsCount()
    
    i = 0
    while True:
        i += 1
        printNow("Beginning iteration #" + str(i) + ".  Calling sqlite ...")
        getNewPaths()
        printNow("Updating paths")
        updatePathsWithNewpaths()
        currentCount = getPathsCount()
        printNow("Done, current path total is: " + str(currentCount))
        if (currentCount == oldCount):
            break
        else:
            oldCount = currentCount

            
    print 'Converged! The average length of the shortest paths is: ', getAveragePathsLength()

findDegreeDistribution()

initializing degrees.  Calling sqlite...
Done, now populating paths.  Calling sqlite...
Done, now getting pathcount.  Calling sqlite...
Beginning iteration #1.  Calling sqlite ...
Updating paths
Done, current path total is: 3817
Beginning iteration #2.  Calling sqlite ...
Updating paths
Done, current path total is: 5542
Beginning iteration #3.  Calling sqlite ...
Updating paths
Done, current path total is: 7117
Beginning iteration #4.  Calling sqlite ...
Updating paths
Done, current path total is: 8549
Beginning iteration #5.  Calling sqlite ...
Updating paths
Done, current path total is: 9850
Beginning iteration #6.  Calling sqlite ...
Updating paths
Done, current path total is: 11025
Beginning iteration #7.  Calling sqlite ...
Updating paths
Done, current path total is: 12107
Beginning iteration #8.  Calling sqlite ...
Updating paths
Done, current path total is: 13099
Beginning iteration #9.  Calling sqlite ...
Updating paths
Done, current path total is: 13951
Beginning iteration #10

In [7]:
# Q2
# Expected output: nodes table populated with pagerank results 
# Note:
# - keeping track of the old PR value helps find convergence, though it's not required in the
#   assignment, it will help you see how close the values end up becoming.
def initPageRank():
    dbQ2.executescript("""
        DROP TABLE IF EXISTS nodes;
        CREATE TABLE nodes (
            node varchar(200) PRIMARY KEY,
            prevpr real,
            currentpr real,
            outdegree integer
        );
    """);
    dbQ2.commit()
    return


def populatePageRank():
    dbQ2.executescript("""
        INSERT INTO nodes
        SELECT
            DISTINCT links.src,
            1,
            1,
            COUNT(links.dst)
        FROM
            links
        GROUP BY
            links.src;
               
        -- corner case #1 nodes with no outgoing edges
        INSERT INTO
            nodes
        SELECT
            DISTINCT links.dst,
            1,
            1,
            0
        FROM
            links
        WHERE
            links.dst NOT IN (
                SELECT node FROM nodes
            );    
            
        -- corner cases #2 nodes with no incoming edges
        UPDATE
            nodes
        SET
            currentpr = 0.15
        WHERE
            nodes.node not in (
                SELECT DISTINCT dst
                FROM links
            );
    """);
    dbQ2.commit()

    return


        
def updatePageRank():

    dbQ2.executescript(""" 
        -- TODO (hint: you might want a helper table to store intermediate results)

        
        
        DROP TABLE IF EXISTS temp;
        CREATE TABLE temp (
            node varchar(200) PRIMARY KEY,
            prevpr real,
            currentpr real,
            outdegree integer
        );
        INSERT INTO temp
        
        
        SELECT dst, nodes.currentpr AS prevpr, num, nodes.outdegree 
        FROM nodes,
        
            (SELECT dst, src, (0.85*SUM(num)+0.15) AS num 
            FROM
            
                (SELECT dst, src, nodes.currentpr/nodes.outdegree AS num 
                FROM nodes,
                
                   (SELECT dst, src, nodes.node AS tgt
                   FROM nodes,
                
                    (SELECT links.src AS src, nodes.node AS dst  
                    FROM links, nodes 
                    WHERE links.dst = nodes.node)
                    
                   WHERE nodes.node = src) 
                    
                WHERE nodes.node = tgt) 
                
            GROUP BY dst) 
            
        WHERE dst = nodes.node;
        
        DROP TABLE IF EXISTS nodes;
        ALTER TABLE temp RENAME TO nodes; 
    """);
    dbQ2.commit()

            
    return



def topPageRank():
    
    dbQ2.executescript("""
    
        DROP VIEW IF EXISTS topPR;
        CREATE VIEW topPR AS
        SELECT *
        FROM nodes
        ORDER BY currentpr DESC
        LIMIT 100;
        
    """);
    dbQ2.commit()

    return

def printTopPageRank():
    # we need to create an cursor object to fetch results to this program
    #    which we didn't need to before since it was just updating the DB
    r = dbQ1.execute("SELECT * FROM topPR;").fetchall()
    for i in r:
        print i
    

def runPageRank(N=10):
    printNow('Initializing page rank')
    initPageRank()
    printNow('Populating PageRank')
    populatePageRank()
    for i in xrange(N):
        printNow('Updating PageRank for iteration {}'.format(i))
        updatePageRank()
        
    printNow('Running top pagerank!')        
    topPageRank()
    printNow('Done! Here are your results')        
    printTopPageRank()
    
    return


runPageRank()

Initializing page rank
Populating PageRank
Updating PageRank for iteration 0
Updating PageRank for iteration 1
Updating PageRank for iteration 2
Updating PageRank for iteration 3
Updating PageRank for iteration 4
Updating PageRank for iteration 5
Updating PageRank for iteration 6
Updating PageRank for iteration 7
Updating PageRank for iteration 8
Updating PageRank for iteration 9
Running top pagerank!
Done! Here are your results
('Latin', 3.54392768882394, 3.75857177114488, 1)
('English_language', 3.644415864015341, 3.162338535500349, 2)
('Spain', 3.7562500319333454, 3.0432170457431003, 1)
('People%27s_Republic_of_China', 3.00045251391133, 2.7003846368246305, 1)
('Fascism', 3.403784759697765, 2.5702261027992397, 1)
('Republic_of_Ireland', 1.5933498262660337, 1.8091684836516846, 1)
('Channel_Islands', 1.4836733173344336, 1.6988767422065199, 0)
('Edward_VIII_of_the_United_Kingdom', 1.9270577345184483, 1.559595850528886, 1)
('Italy', 1.6918746197587715, 1.5043473523261286, 2)
('Franz_Schu

In [9]:
# you should have ran the two previous cells before running the test!
# make sure also that you have set the db to part2test.db!
! ./test.sh

Error: no such table: paths
ERROR q1! See your_output/q1.csv
Error: no such table: topPR
ERROR q2! See your_output/q2.csv
